In [2]:
# this is to deduplicate matches that we already have. if no match numbers, use 
# deduplicate function in match_pipeline

# note: this was made before we created the unsure column so it doesn't take that into account 

import pandas as pd
import numpy as np

In [117]:
master = pd.read_excel('current_master_6_17_2020.xlsx')
matches = pd.read_csv('20200609_result-2.csv')

In [5]:
matches.head()

,Unnamed: 0,calculated_birth_year,census_year,county,first_name,full_name,last_name,lat,long,marital_status,...,profession,race,religion,roll_or_sheet,sex,state_or_province,street,unique_id,ward,dup_pair
0,0,1818.0,1880.0,unknown,NaN,------,NaN,0.0,3046507000.0,unknown,...,NaN,white,unknown,9.0,m,unknown,unknown,2390.0,unknown,0
1,1,1819.0,1880.0,unknown,NaN,------,NaN,0.0,3046507000.0,unknown,...,NaN,black,unknown,9.0,m,unknown,unknown,2391.0,unknown,0
2,2,1857.0,1880.0,unknown,NaN,------,NaN,0.0,4112398000.0,unknown,...,laborer,mixed,unknown,2.0,m,unknown,unknown,2377.0,unknown,1
3,3,1857.0,1880.0,unknown,NaN,------,NaN,0.0,3046507000.0,unknown,...,worksonfarm,black,unknown,9.0,m,unknown,unknown,2395.0,unknown,1
4,4,1857.0,1880.0,unknown,NaN,------,NaN,0.0,4112398000.0,unknown,...,laborer,mixed,unknown,2.0,m,unknown,unknown,2377.0,unknown,2


In [118]:
matches = matches.sort_values(by = "dup_pair")

In [119]:
pairs_to_delete = []
for i in range(0, len(matches.index), 2):
    # pull out unique ids and match number
    first_id = matches.loc[i, 'unique_id']
    second_id = matches.loc[i+1, 'unique_id']
    pair_num = matches.loc[i, 'dup_pair']
    
    # find those ids in master and pull match numbers
    match_id_1 = master[master['unique_id'] == first_id]['Joint ID for Matched Records']
    match_id_2 = master[master['unique_id'] == second_id]['Joint ID for Matched Records']
    
    # if they have the same match number, they're already a match 
    if np.array_equal(match_id_1, match_id_2):
        pairs_to_delete.append(pair_num)
        
# drop duplicates 
matches.drop(matches[matches['dup_pair'].isin(pairs_to_delete)].index, inplace = True) 
print(str(len(pairs_to_delete)) + " pairs dropped")


18 pairs dropped


In [120]:
matches = matches.reset_index(drop = True)
matches = matches.sort_values(by = "dup_pair")

In [121]:
# create new file of matches to check that aren't duplicates
new_file = pd.DataFrame(columns = master.columns)
for i in range(len(matches.index)):
    unique_id = matches.loc[i, 'unique_id']
    new_file = new_file.append(master[master['unique_id'] == unique_id], ignore_index = True)
    
new_file['new_match_id'] = matches['dup_pair']
# can use create_match_numbers once matches are approved to assign correct match numbers for the master

In [123]:
new_file.head(20)

,unique_id,source,Census.Year,State/Province,County,Place,Household Joint ID,Joint ID for Matched Records,Last.Name,First.Name,...,Rank (Military),Enlistment Date,Enlistment Place,Date Mustered Out,Year of this Record,Last Name MATCH,First Name Match,Census Year Match,Total of Matches,new_match_id
0,2390.0,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5900,1230,NaN,NaN,NaN,0,0,0,FALSE,0
1,2391.0,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5900,1230,NaN,NaN,NaN,0,0,0,FALSE,0
2,2377.0,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5900,1230,NaN,NaN,NaN,100,0,100,False,1
3,2395.0,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5900,1230,NaN,NaN,NaN,100,100,100,True,1
4,2377.0,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5900,1230,NaN,NaN,NaN,100,0,100,False,2
5,2396.0,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,4430,801,NaN,NaN,NaN,100,0,100,FALSE,2
6,2395.0,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5900,1230,NaN,NaN,NaN,100,100,100,True,3
7,2396.0,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,4430,801,NaN,NaN,NaN,100,0,100,FALSE,3
8,2440.0,20271,1920,ME,PENOBSCOT,BANGOR,NaN,NaN,???,ERNEST,...,NaN,NaN,NaN,NaN,NaN,100,100,100,True,4
9,2441.0,20319,1920,ME,PENOBSCOT,BANGOR,NaN,NaN,???,ERNEST,...,NaN,NaN,NaN,NaN,NaN,0,0,100,False,4


In [124]:
new_file.to_csv(r'/Users/seb2244/Desktop/migration_project/dedup_20200609.csv', index = False)